In [120]:
import pandas as pd
import numpy as np
from numpy import matmul
from scipy import stats
from sklearn import metrics
from numpy.linalg import inv
import random
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_classif, SelectKBest, chi2
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored, cumulative_dynamic_auc


In [95]:
def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]

def vectorized_gaussian_logpdf(X, means, covariances):
    # """
    # Compute log N(x_i; mu_i, sigma_i) for each x_i, mu_i, sigma_i
    # Args:
    #     X : shape (d)
    #         Data points
    #     means : shape (self.N, d)
    #         Mean vectors
    #     covariances : shape (self.N, d)
    #         Diagonal covariance matrices
    # Returns:
    #     logpdfs : shape (n,)
    #         Log probabilities
    # """
        d = X.shape[0]
        constant = d * np.log(2 * np.pi)
        log_determinants = np.log(np.prod(covariances, axis=1))
        deviations = X - means
        inverses = 1 / covariances
        return -0.5 * (constant + log_determinants + np.sum(deviations * inverses * deviations, axis=1))

def vectorized_norm_loglikelihood(X, means, covariances, delta):
    #  '''
    #  Compute log Norm(X_i, mu_i, sigma_i) for each x_i, mu_i, sigma_i
    #  Args:
    #      X: shape (n,)
    #      means: shape (n,d)
    #      covariances (d,)
    #  Returns: logsf: shape (d,)
    #  '''
    loglikelihood = [delta*np.log(stats.norm.pdf(x = X, loc=means[:, i], scale=np.sqrt(covariances[i]))) + (1-delta)*np.log(stats.norm.sf(x = X, loc=means[:, i], scale=np.sqrt(covariances[i]))) for i in range(means.shape[1])]
    loglikelihood = np.array(loglikelihood).sum(axis=1)
    return loglikelihood

In [96]:
Z_Y = []
Z_X = []
BETA_Y = []
TAU_Y = []
BETA_X = []
TAU_X = []
data_train = []
ind_train = [0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16]
ind_test = [7,17,18,19]
for i in ind_train:
    dat = pd.read_csv(f'C:/Users/ndong/OneDrive - Texas Tech University/Documents/Scalable Survival-DP Package/Simulation/Original Copy of Simulation/Simulation 3 - small std/simulated data 3_{i+1}.csv', index_col=0)
    z_y = pd.read_csv(f'C:/Users/ndong/OneDrive - Texas Tech University/Documents/Scalable Survival-DP Package/Simulation/Original Copy of Simulation/Simulation 3 - small std/3_{i+1}_results/simulation_3_{i+1}_Z_Y_records.csv', index_col=0)
    z_x = pd.read_csv(f'C:/Users/ndong/OneDrive - Texas Tech University/Documents/Scalable Survival-DP Package/Simulation/Original Copy of Simulation/Simulation 3 - small std/3_{i+1}_results/simulation_3_{i+1}_Z_X_records.csv', index_col=0)
    beta_y = pd.read_csv(f'C:/Users/ndong/OneDrive - Texas Tech University/Documents/Scalable Survival-DP Package/Simulation/Original Copy of Simulation/Simulation 3 - small std/3_{i+1}_results/simulation_3_{i+1}_BETA_Y_records.csv', index_col=0)
    tau_y = pd.read_csv(f'C:/Users/ndong/OneDrive - Texas Tech University/Documents/Scalable Survival-DP Package/Simulation/Original Copy of Simulation/Simulation 3 - small std/3_{i+1}_results/simulation_3_{i+1}_TAU_Y_records.csv', index_col=0)
    beta_x = pd.read_csv(f'C:/Users/ndong/OneDrive - Texas Tech University/Documents/Scalable Survival-DP Package/Simulation/Original Copy of Simulation/Simulation 3 - small std/3_{i+1}_results/simulation_3_{i+1}_BETA_X_records.csv', index_col=0)
    tau_x = pd.read_csv(f'C:/Users/ndong/OneDrive - Texas Tech University/Documents/Scalable Survival-DP Package/Simulation/Original Copy of Simulation/Simulation 3 - small std/3_{i+1}_results/simulation_3_{i+1}_TAU_X_records.csv', index_col=0)
    Z_Y.append(z_y)
    Z_X.append(z_x)
    BETA_Y.append(beta_y)
    TAU_Y.append(tau_y)
    BETA_X.append(beta_x)
    TAU_X.append(tau_x)
    data_train.append(dat)


In [97]:
data_test=[]
for i in ind_test:
    dat = pd.read_csv(f'C:/Users/ndong/OneDrive - Texas Tech University/Documents/Scalable Survival-DP Package/Simulation/Original Copy of Simulation/Simulation 3 - small std/simulated data 3_{i+1}.csv', index_col=0)
    data_test.append(dat)

In [98]:
Training = pd.concat(data_train, ignore_index=True)
true_train_z_y = Training['z_y']
true_train_y = Training['Y']
Testing = pd.concat(data_test, ignore_index=True)
true_test_z_y = Testing['z_y']
true_test_y = Testing['Y']
delta_test = Testing['c_y']
Testing1 = Testing.drop(columns=['z_x', 'z_y', 'c_y', 'y_mu', 'y_sigma', 'Y'])
X_name = Testing1.columns

In [136]:
# Agg_z_y = [[Z_Y[i].copy() for i in range(16)] for j in range(50)]
# Agg_z_x = [[Z_X[i].copy() for i in range(16)] for j in range(50)]
# Agg_beta_y = [[BETA_Y[i].copy() for i in range(16)] for j in range(50)]
# Agg_tau_y = [[TAU_Y[i].copy() for i in range(16)] for j in range(50)]

# n_subsets_cluster_y = [[Z_Y[j].iloc[:,i].nunique() for j in range(16)] for i in range(50)]
# print(n_subsets_cluster_y)
# n_subsets_cluster_x = [[Z_X[j].iloc[:,i].nunique() for j in range(16)] for i in range(50)]
# random.seed(120)
# reference_subset = random.choices(range(16),k=50)
# print(reference_subset)

# for t in range(50):
#     i = reference_subset[t]
#     print(i)
#     for j in range(50):
#         target_z_y = Z_Y[i].iloc[:,j]
#         n_N_y = target_z_y.value_counts(sort=False).sort_index()
#         target_z_x = Z_X[i].iloc[:,j]
#         beta_y = BETA_Y[i]
#         name_beta_y = Filter(beta_y.columns, [f'in_{j+1}_iteration'])
#         target_beta_y = beta_y[name_beta_y].values
#         tau_y = TAU_Y[i]
#         name_tau_y = Filter(tau_y.columns, [f'in_{j+1}_iteration'])
#         target_tau_y = tau_y[name_tau_y].values
#         beta_x = BETA_X[i]
#         name_beta_x = Filter(beta_x.columns, [f'in_{j+1}_iteration'])
#         target_beta_x = beta_x[name_beta_x].values
#         tau_x = TAU_X[i]
#         name_tau_x = Filter(tau_x.columns, [f'in_{j+1}_iteration'])
#         target_tau_x = tau_x[name_tau_x].values
#         for k in range(16):
#             n_n = n_subsets_cluster_y[j][k]
#             print(f'the {k+1}th subset in the {j+1}th iteration')
#             if k != i:
#                 DAT = data_train[k]
#                 for s in range(n_n):
#                     ddd = DAT[Z_Y[k][f'z_y_{j+1}_iteration'] == s]
#                     print(ddd.shape)
#                     X = ddd[X_name].values
#                     delta = ddd['c_y'].values
#                     y = ddd['Y'].values
#                     print(y.shape)
#                     mean_y = matmul(X, target_beta_y)
#                     print(mean_y.shape)
#                     likelihood_y = vectorized_norm_loglikelihood(y, mean_y, target_tau_y[0], delta)
#                     print(likelihood_y)
#                     likelihood_y = np.nan_to_num(likelihood_y, nan=-np.inf)
#                     likelihood_x = [np.exp(vectorized_gaussian_logpdf(X[t], target_beta_x.T, target_tau_x.T)) for t in range(X.shape[0])]
#                     likelihood_x = np.array(likelihood_x).sum(axis=0)
#                     true_z_y = np.argmax(likelihood_y)
#                     print(true_z_y)
#                     true_z_x = np.argmax(likelihood_x)
#                     Agg_z_y[t][k].loc[ddd.index, f'z_y_{j+1}_iteration'] = true_z_y
#                     Agg_z_x[t][k].loc[ddd.index, f'z_x_{j+1}_iteration'] = true_z_x

In [ ]:
Agg_z_y = [Z_Y[i].copy() for i in range(16)]
Agg_z_x = [Z_X[i].copy() for i in range(16)]

n_subsets_cluster_y = [[Z_Y[j].iloc[:,i].nunique() for j in range(16)] for i in range(50)]
# print(n_subsets_cluster_y)
n_subsets_cluster_x = [[Z_X[j].iloc[:,i].nunique() for j in range(16)] for i in range(50)]
random.seed(10)
reference_subset = random.choices(range(16),k=50)
# print(reference_subset)
for t in range(50):
    i = reference_subset[t]
    # print(i)
    target_z_y = Z_Y[i].iloc[:,t]
    n_N_y = target_z_y.value_counts(sort=False).sort_index()
    target_z_x = Z_X[i].iloc[:,t]
    beta_y = BETA_Y[i]
    name_beta_y = Filter(beta_y.columns, [f'in_{t+1}_iteration'])
    target_beta_y = beta_y[name_beta_y].values
    tau_y = TAU_Y[i]
    name_tau_y = Filter(tau_y.columns, [f'in_{t+1}_iteration'])
    target_tau_y = tau_y[name_tau_y].values
    beta_x = BETA_X[i]
    name_beta_x = Filter(beta_x.columns, [f'in_{t+1}_iteration'])
    target_beta_x = beta_x[name_beta_x].values
    tau_x = TAU_X[i]
    name_tau_x = Filter(tau_x.columns, [f'in_{t+1}_iteration'])
    target_tau_x = tau_x[name_tau_x].values
    for k in range(16):
        n_n = n_subsets_cluster_y[t][k]
        # print(f'the {k+1}th subset in the {t+1}th iteration')
        if k != i:
            DAT = data_train[k]
            for s in range(n_n):
                ddd = DAT[Z_Y[k][f'z_y_{t+1}_iteration'] == s]
                # print(ddd.shape)
                X = ddd[X_name].values
                delta = ddd['c_y'].values
                y = ddd['Y'].values
                # print(y.shape)
                mean_y = matmul(X, target_beta_y)
                # print(mean_y.shape)
                likelihood_y = vectorized_norm_loglikelihood(y, mean_y, target_tau_y[0], delta)
                # print(likelihood_y)
                likelihood_y = np.nan_to_num(likelihood_y, nan=-np.inf)
                likelihood_x = [np.exp(vectorized_gaussian_logpdf(X[t], target_beta_x.T, target_tau_x.T)) for t in range(X.shape[0])]
                likelihood_x = np.array(likelihood_x).sum(axis=0)
                true_z_y = np.argmax(likelihood_y)
                # print(true_z_y)
                true_z_x = np.argmax(likelihood_x)
                Agg_z_y[k].loc[ddd.index, f'z_y_{t+1}_iteration'] = true_z_y
                Agg_z_x[k].loc[ddd.index, f'z_x_{t+1}_iteration'] = true_z_x


In [100]:
train_RI = []
train_MI = []
train_ARI = []
train_AMI = []
train_HOMO = []
train_COM = []
train_VM = []
train_FM = []
AGG_Z_Y = pd.concat(Agg_z_y, ignore_index=True)
for j in range(AGG_Z_Y.shape[1]):
    ri = metrics.rand_score(true_train_z_y, AGG_Z_Y.iloc[:,j])
    mi = metrics.mutual_info_score(true_train_z_y, AGG_Z_Y.iloc[:,j])
    ari = metrics.adjusted_rand_score(true_train_z_y,AGG_Z_Y.iloc[:,j])
    ami = metrics.adjusted_mutual_info_score(true_train_z_y, AGG_Z_Y.iloc[:,j])
    hcv = metrics.homogeneity_completeness_v_measure(true_train_z_y, AGG_Z_Y.iloc[:,j])
    fm = metrics.fowlkes_mallows_score(true_train_z_y, AGG_Z_Y.iloc[:,j])
    train_RI.append(ri)
    train_MI.append(mi)
    train_ARI.append(ari)
    train_AMI.append(ami)
    train_HOMO.append(hcv[0])
    train_COM.append(hcv[1])
    train_VM.append(hcv[2])
    train_FM.append(fm)

    
    



In [102]:
pd.DataFrame({'Train-RI': [np.array(train_RI).mean(), np.array(train_RI).std()],'Train-MI': [np.array(train_MI).mean(), np.array(train_MI).std()],'Train-ARI': [np.array(train_ARI).mean(), np.array(train_ARI).std()], 'Train-AMI': [np.array(train_AMI).mean(), np.array(train_AMI).std()],'Train-HOMO': [np.array(train_HOMO).mean(), np.array(train_HOMO).std()], 'Train-COM': [np.array(train_COM).mean(), np.array(train_COM).std()], 'Train-VM': [np.array(train_VM).mean(), np.array(train_VM).std()], 'Train-FM': [np.array(train_FM).mean(), np.array(train_FM).std()]}, index=['mean', 'std'])

,Train-RI,Train-MI,Train-ARI,Train-AMI,Train-HOMO,Train-COM,Train-VM,Train-FM
mean,0.860236,0.781942,0.704578,0.745203,0.711771,0.790145,0.745267,0.818175
std,0.060733,0.111932,0.114162,0.064951,0.101887,0.033299,0.064932,0.058702


In [ ]:
# Test clustering:
Agg_z_y_test = pd.DataFrame(np.ones((Testing1.shape[0], 50)), columns=Z_Y[0].columns)
Agg_z_x_test = pd.DataFrame(np.ones((Testing1.shape[0], 50)), columns=Z_X[0].columns)
print(len(n_subsets_cluster_y))
for t in range(50):
    i = reference_subset[t]
    n_n = n_subsets_cluster_y[t][i]
    print(n_n)
    target_z_y = Z_Y[i].iloc[:,t]
    n_N_y = target_z_y.value_counts(sort=False).sort_index()
    target_z_x = Z_X[i].iloc[:,t]
    beta_y = BETA_Y[i]
    name_beta_y = Filter(beta_y.columns, [f'in_{t+1}_iteration'])
    target_beta_y = beta_y[name_beta_y].values
    target_beta_y = target_beta_y[:,:n_n]
    tau_y = TAU_Y[i]
    name_tau_y = Filter(tau_y.columns, [f'in_{t+1}_iteration'])
    target_tau_y = tau_y[name_tau_y].values
    target_tau_y = target_tau_y[:,:n_n]
    beta_x = BETA_X[i]
    name_beta_x = Filter(beta_x.columns, [f'in_{t+1}_iteration'])
    target_beta_x = beta_x[name_beta_x].values
    tau_x = TAU_X[i]
    name_tau_x = Filter(tau_x.columns, [f'in_{t+1}_iteration'])
    target_tau_x = tau_x[name_tau_x].values
    n_N_x = []
    print(target_beta_y.shape, target_beta_x.shape)
    for s in range(n_subsets_cluster_x[t][i]):
        group = target_z_y[target_z_x == s]
        n_m = np.zeros(n_N_y.shape[0])
        n_m_effective = group.value_counts(sort=False).sort_index()
        n_m[n_m_effective.index] = n_m_effective
        n_N_x.append(n_m)
    n_N_x = np.array(n_N_x).T
    print(n_N_x)
    for j in range(Testing1.shape[0]):
        y = true_test_y[j]
        x = Testing1.loc[j, ].values
        delta = delta_test[j]
        mean_y = matmul(x.reshape(1,-1), target_beta_y)
        likelihood_x = vectorized_gaussian_logpdf(x, target_beta_x.T, target_tau_x.T)
        pred_z_x = np.argmax(likelihood_x)
        # ind_x = []
        # for k in range(n_n):
        #     zzx = target_z_x[target_z_y == k]
        #     ind_x.append(zzx.iloc[0])
        likelihood_x = np.exp(likelihood_x)
        # print(likelihood_x.shape)
        likelihood_x = matmul(n_N_x, likelihood_x)
        likelihood_x = likelihood_x[:n_n]
        # print(likelihood_x.shape)
        likelihood_y = (stats.norm.pdf(y, loc=mean_y[0], scale=np.sqrt(target_tau_y[0]))**delta) * (stats.norm.sf(y, loc=mean_y[0], scale=np.sqrt(target_tau_y[0]))**(1-delta))
        pred_z_y = np.argmax(likelihood_y*likelihood_x)
        # print(np.argmax(likelihood_y+likelihood_x), true_test_z_y[j])
        Agg_z_y_test.iloc[j, t] = pred_z_y
        Agg_z_x_test.iloc[j, t] = pred_z_x


In [106]:
test_RI = []
test_MI = []
test_ARI = []
test_AMI = []
test_HOMO = []
test_COM = []
test_VM = []
test_FM = []
for j in range(43):
    ri = metrics.rand_score(true_test_z_y, Agg_z_y_test.iloc[:,j])
    mi = metrics.mutual_info_score(true_test_z_y, Agg_z_y_test.iloc[:,j])
    ari = metrics.adjusted_rand_score(true_test_z_y,Agg_z_y_test.iloc[:,j])
    ami = metrics.adjusted_mutual_info_score(true_test_z_y, Agg_z_y_test.iloc[:,j])
    hcv = metrics.homogeneity_completeness_v_measure(true_test_z_y, Agg_z_y_test.iloc[:,j])
    fm = metrics.fowlkes_mallows_score(true_test_z_y, Agg_z_y_test.iloc[:,j])
    test_RI.append(ri)
    test_MI.append(mi)
    test_ARI.append(ari)
    test_AMI.append(ami)
    test_HOMO.append(hcv[0])
    test_COM.append(hcv[1])
    test_VM.append(hcv[2])
    test_FM.append(fm)

In [107]:
pd.DataFrame({'Test-RI': [np.array(test_RI).mean(), np.array(test_RI).std()],'Test-MI': [np.array(test_MI).mean(), np.array(test_MI).std()],'Test-ARI': [np.array(test_ARI).mean(), np.array(test_ARI).std()], 'Test-AMI': [np.array(test_AMI).mean(), np.array(test_AMI).std()],'Test-HOMO': [np.array(test_HOMO).mean(), np.array(test_HOMO).std()], 'Test-COM': [np.array(test_COM).mean(), np.array(test_COM).std()], 'Test-VM': [np.array(test_VM).mean(), np.array(test_VM).std()], 'Test-FM': [np.array(test_FM).mean(), np.array(test_FM).std()]}, index=['mean', 'std'])

,Test-RI,Test-MI,Test-ARI,Test-AMI,Test-HOMO,Test-COM,Test-VM,Test-FM
mean,0.902335,0.905221,0.785678,0.832384,0.823992,0.842278,0.832544,0.861297
std,0.059464,0.118482,0.129125,0.093650,0.107850,0.081343,0.093562,0.082607


In [ ]:
Agg_beta_y = [BETA_Y[i].copy() for i in range(16)]
Agg_tau_y = [TAU_Y[i].copy() for i in range(16)]


for t in range(50):
    i = reference_subset[t]
    for k in range(16):
        n_n = n_subsets_cluster_y[t][k]
        if k != i:
            for s in range(n_n): #s is the label in original subsets
                ind = Z_Y[k][Z_Y[k][f'z_y_{t+1}_iteration'] == s].index
                agged_z_y = Agg_z_y[k].loc[ind[0], f'z_y_{t+1}_iteration']
                Agg_beta_y[k][f'beta_y_cluster{agged_z_y+1}_in_{t+1}_iteration'] = BETA_Y[k][f'beta_y_cluster{s+1}_in_{t+1}_iteration']
                Agg_tau_y[k][f'tau_y_cluster{agged_z_y+1}_in_{t+1}_iteration'] = TAU_Y[k][f'tau_y_cluster{s+1}_in_{t+1}_iteration']

In [ ]:
#####To be rewrite


Average_beta_over_iterations_name = BETA_Y[0].columns
Average_beta_over_iterations = []
Average_tau_over_iterations_name = TAU_Y[0].columns
Average_tau_over_iterations = []
for i in range(50):
    # alter_beta_over_iter = []
    # alter_tau_over_iter = []
    for k in range(3):
        alter_beta_over_iter_cluster = []
        alter_tau_over_iter_cluster = []
        for j in range(16):
            alter_beta_over_iter_cluster.append(Agg_beta_y[j][f'beta_y_cluster{k+1}_in_{i+1}_iteration'])
            alter_tau_over_iter_cluster.append(Agg_tau_y[j][f'tau_y_cluster{k+1}_in_{i+1}_iteration'])
        alter_beta_over_iter_cluster = np.array(alter_beta_over_iter_cluster).mean(axis=0)
        alter_tau_over_iter_cluster = np.array(alter_tau_over_iter_cluster).mean()
        Average_beta_over_iterations.append(alter_beta_over_iter_cluster)
        Average_tau_over_iterations.append(alter_tau_over_iter_cluster)
Average_beta_along_iterations = pd.DataFrame(np.array(Average_beta_over_iterations).T, columns=Average_beta_over_iterations_name)
Average_tau_along_iterations = pd.DataFrame(np.array(Average_tau_over_iterations).reshape(1,-1), columns=Average_tau_over_iterations_name)
print(Average_beta_along_iterations.shape)

In [ ]:
Average_beta_over_subsets_name = []
Average_beta_over_subsets = []
Average_tau_over_subsets_name = []
Average_tau_over_subsets = []
for i in ind_train:
    for j in range(3):
        Average_beta_over_subsets_name.append(f'beta_y_cluster{j+1}_in_{i+1}_subset')
        Average_tau_over_subsets_name.append(f'tau_y_cluster{j+1}_in_{i+1}_subset')

for i in range(16):
    # alter_beta_over_iter = []
    # alter_tau_over_iter = []
    for k in range(3):
        alter_beta_over_sub_cluster = []
        alter_tau_over_sub_cluster = []
        for j in range(50):
            alter_beta_over_sub_cluster.append(Agg_beta_y[i][f'beta_y_cluster{k+1}_in_{j+1}_iteration'])
            alter_tau_over_sub_cluster.append(Agg_tau_y[i][f'tau_y_cluster{k+1}_in_{j+1}_iteration'])
        alter_beta_over_sub_cluster = np.array(alter_beta_over_iter_cluster).mean(axis=0)
        alter_tau_over_sub_cluster = np.array(alter_tau_over_iter_cluster).mean()
        Average_beta_over_subsets.append(alter_beta_over_iter_cluster)
        Average_tau_over_subsets.append(alter_tau_over_iter_cluster)
Average_beta_along_subsets = pd.DataFrame(np.array(Average_beta_over_subsets).T, columns=Average_beta_over_subsets_name)
Average_tau_along_subsets = pd.DataFrame(np.array(Average_tau_over_subsets).reshape(1,-1), columns=Average_tau_over_subsets_name)
print(Average_beta_along_subsets.shape)

In [146]:
preds_time_stamp = [365*i for i in [1, 2, 3, 5, 7, 9]]
SCORE_train = []
SCORE_test = []
PRC_train = []
PRC_test = []

for k in preds_time_stamp:
    Score_train = []
    Score_test = []
    Prc_train = []
    Prc_test = []
    for t in range(50):
        i = reference_subset[t]
        z_y_train = AGG_Z_Y.iloc[:,t]
        z_y_test = Agg_z_y_test.iloc[:,t]
        beta_y = BETA_Y[i]
        name_beta_y = Filter(beta_y.columns, [f'in_{t+1}_iteration'])
        target_beta_y = beta_y[name_beta_y].values
        tau_y = TAU_Y[i]
        name_tau_y = Filter(tau_y.columns, [f'in_{t+1}_iteration'])
        target_tau_y = tau_y[name_tau_y].values
        score_train = []
        score_test = []
        prc_ttrain = []
        prc_ttest = []
        for j in range(3):
            dat_train = Training[z_y_train == j]
            dat_test = Testing[z_y_test == j]
            times_train = np.unique(np.exp(dat_train['Y']))
            times_test = np.unique(np.exp(dat_test['Y']))
            y_train1 = Surv.from_arrays(dat_train['c_y'], np.exp(dat_train['Y']))
            y_test1 = Surv.from_arrays(dat_test['c_y'], np.exp(dat_test['Y']))
            if times_train.max() > k and times_train.min() < k and times_test.max() > k and times_test.min() < k:
                X_train = dat_train[X_name].values
                X_test = dat_test[X_name].values
                delta_train = dat_train['c_y']
                delta_test = dat_test['c_y']
                beta = target_beta_y[:,j] 
                mean_y_train = matmul(X_train, beta)
                mean_y_test = matmul(X_test, beta)
                tau = target_tau_y[:,j]
                # pdf_train = stats.norm.pdf(k, loc=mean_y_train, scale=np.sqrt(tau[0]))
                # pdf_test = stats.norm.pdf(k, loc=mean_y_test, scale=np.sqrt(tau[0]))
                # cdf_train = stats.norm.cdf(k, loc=mean_y_train, scale=np.sqrt(tau[0]))
                # cdf_test = stats.norm.cdf(k, loc=mean_y_test, scale=np.sqrt(tau[0]))
                sf_train = stats.norm.cdf(np.log(k), loc=mean_y_train, scale=np.sqrt(tau[0]))
                sf_test = stats.norm.cdf(np.log(k), loc=mean_y_test, scale=np.sqrt(tau[0]))
                auroc_train = cumulative_dynamic_auc(y_train1, y_train1, sf_train, k)
                auroc_test = cumulative_dynamic_auc(y_train1, y_test1, sf_test, k)
                # h_train = pdf_train/sf_train
                # h_test = pdf_test/sf_test
                # H_train = -np.log(sf_train)
                # H_test = -np.log(sf_train)
                score_train.append(auroc_train[0])
                score_test.append(auroc_test[0])
                precision_train, recall_train, thresholds_train = metrics.precision_recall_curve(delta_train, sf_train)
                precision_test, recall_test, thresholds_test = metrics.precision_recall_curve(delta_test,
                                                                                                sf_test)
                prc_train = pd.DataFrame({'precision_train': precision_train, 'recall_train': recall_train})
                prc_test = pd.DataFrame({'precision_test': precision_test, 'recall_test': recall_test})
                prc_train.sort_values(by='precision_train', inplace=True)
                prc_test.sort_values(by='precision_test', inplace=True)
                auprc_train = metrics.auc(recall_train, precision_train)
                auprc_test = metrics.auc(recall_test, precision_test)
                prc_ttrain.append(auprc_train)
                prc_ttest.append(auprc_test)
        Score_train.append(np.mean(score_train))
        Score_test.append(np.mean(score_test))
        Prc_train.append(np.mean(auprc_train))
        Prc_test.append(np.mean(auprc_test))
    SCORE_train.append(Score_train)
    SCORE_test.append(Score_test)
    PRC_train.append(Prc_train)
    PRC_test.append(Prc_test)
SCORE_train = np.array(SCORE_train)
SCORE_test = np.array(SCORE_test)
PRC_train = np.array(PRC_train)
PRC_test = np.array(PRC_test)
print(PRC_train.shape)
# pd.DataFrame({'AUROC-Training':[np.mean(SCORE_train), np.std(SCORE_train)], 'AUROC'})

(6, 50)


In [141]:
SCORE_train.mean(axis=1)


array([0.99889105, 0.89649387, 0.85904661, 0.94074486, 0.95445491,
       0.96175068 ])

In [142]:
SCORE_test.mean(axis=1)


array([0.99796804, 0.88915669, 0.84856715, 0.91294391, 0.91992174,
       0.94783624])

In [147]:
PRC_train.mean(axis=1)


array([0.32721594, 0.60649115, 0.73964618 , 0.95693547, 0.97987916,
       0.99903519])

In [148]:
PRC_test.mean(axis=1)

array([0.33051496, 0.59039158, 0.73939994, 0.95696487, 0.98384205,
       0.99920564])

In [118]:
CI_score_train = []
CI_score_test = []
for t in range(50):
    i = reference_subset[t]
    z_y_train = AGG_Z_Y.iloc[:,t]
    z_y_test = Agg_z_y_test.iloc[:,t]
    beta_y = BETA_Y[i]
    name_beta_y = Filter(beta_y.columns, [f'in_{t+1}_iteration'])
    target_beta_y = beta_y[name_beta_y].values
    tau_y = TAU_Y[i]
    name_tau_y = Filter(tau_y.columns, [f'in_{t+1}_iteration'])
    target_tau_y = tau_y[name_tau_y].values
    ci_score_train = []
    ci_score_test = []
    for j in range(3):
        Z_y_t = Agg_z_y[i].loc[Agg_z_y[i][f'z_y_{t+1}_iteration'] == j,]
        dat_train = Training[z_y_train == j]
        dat_test = Testing[z_y_test == j]
        X_train = dat_train[X_name].values
        X_test = dat_test[X_name].values
        y_train = np.exp(dat_train['Y'])
        y_test = np.exp(dat_test['Y'])
        delta_train = dat_train['c_y']
        delta_test = dat_test['c_y']
        beta = target_beta_y[:,j]
        mean_y_train = matmul(X_train, beta)
        mean_y_test = matmul(X_test, beta)
        tau = target_tau_y[:,j]
        # pdf_train = stats.norm.pdf(y_train, loc=matmul(X_train, target_beta_y[:,j]), scale=np.sqrt(tau[0]))
        # pdf_test = stats.norm.pdf(y_test, loc=matmul(X_test, target_beta_y[:,j]), scale=np.sqrt(tau[0]))
        # sf_train = stats.norm.sf(y_train, loc=matmul(X_train, target_beta_y[:,j]), scale=np.sqrt(tau[0]))
        # sf_test = stats.norm.sf(y_test, loc=matmul(X_test, target_beta_y[:,j]), scale=np.sqrt(tau[0]))
        # h_train = pdf_train/sf_train
        # h_test = pdf_test/sf_test
        # H_train = -np.log(sf_train)
        # H_test = -np.log(sf_train)
        citrain = concordance_index_censored(delta_train.astype(bool), y_train, -mean_y_train)
        citest = concordance_index_censored(delta_test.astype(bool), y_test, -mean_y_test)
        ci_score_train.append(citrain)
        ci_score_test.append(citest)
    CI_score_train.append(ci_score_train)
    CI_score_test.append(ci_score_test)

In [119]:
CItrain = []
CItest = []
for i in range(50):
    train_correct = []
    train_incorrect = []
    test_correct = []
    test_incorrect = []
    for j in range(3):
        est_train = CI_score_train[i][j]
        est_test = CI_score_test[i][j]
        correct_train = est_train[1]
        incorrect_train = est_train[2]
        correct_test = est_test[1]
        incorrect_test = est_test[2]
        train_correct.append(correct_train)
        train_incorrect.append(incorrect_train)
        test_correct.append(correct_test)
        test_incorrect.append(incorrect_test)
    citrain = sum(train_correct)/(sum(train_correct) + sum(train_incorrect))
    citest = sum(test_correct)/(sum(test_correct) + sum(test_incorrect))
    CItrain.append(citrain)
    CItest.append(citest)

pd.DataFrame({'C-index-train':[np.mean(CItrain), np.std(CItrain)], 'C-index-test':[np.mean(CItest), np.std(CItest)]}, index=['mean', 'std'])
        
        

,C-index-train,C-index-test
mean,0.820680,0.829405
std,0.024989,0.024662


2642